In [1]:
import argparse
import multiprocessing

import pandas as pd
import numpy as np
import torch
from torch import Tensor
from torch_geometric.typing import Adj, OptTensor
from torch_scatter import scatter_add
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import matplotlib.pyplot as plt
from scipy.optimize import minimize


from imputers.feature_propgate import feature_propagation
from imputers.my_imputer2 import FPGCN 

DATA_PATHS = {
    "shenzhen": {"feat": "data/sz_speed.csv", "adj": "data/sz_adj.csv"},
    "losloop": {"feat": "data/los_speed.csv", "adj": "data/los_adj.csv"},
}
def pow_A(adj, k):
    # eigen decomposition
    eps = 1e-10
    adj = adj + eps * torch.eye(adj.shape[0]).to(adj.device)
    eigenvalues, eigenvectors = torch.eig(adj, eigenvectors=True)
    # raise A to the power k
    eigenvalues = eigenvalues[:, 0] ** k
    

data = "shenzhen"
# data = "losloop"
missing_rate = 0.1
mat = pd.read_csv(DATA_PATHS[data]["feat"]).to_numpy() 
adj = pd.read_csv(DATA_PATHS[data]["adj"], header=None).to_numpy()
mat = torch.tensor(mat).float()
adj = torch.tensor(adj)


edge_index = torch.nonzero(adj, as_tuple=False).t().contiguous()
edge_weight = adj[edge_index[0], edge_index[1]].float()
mat = mat.T
mask = torch.bernoulli(torch.Tensor([1 - missing_rate]).repeat(mat.shape)).bool()

2023-05-13 18:39:21.856556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 18:39:27.744605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

def get_product_graph(mat, mask, T):
    N = mat.shape[0]
    mat_T = mat[:, :T]
    mask_T = mask[:, :T]
    nodes = torch.arange(N) 

    edge_index_T = []
    for i in range(T):
        cur_edge_index = edge_index + i * N
        edge_index_T.append(cur_edge_index)

    for i in range(T-1):
        cur_edge_index = torch.stack([nodes + i * N, nodes + (i + 1) * N], dim=0)
        edge_index_T.append(cur_edge_index)
        cur_edge_index = torch.stack([nodes + (i + 1) * N, nodes + i * N], dim=0)
        edge_index_T.append(cur_edge_index)
    
    edge_index_T = torch.cat(edge_index_T, dim=1)
    edge_weight_T = torch.ones(edge_index_T.shape[1])
    mat_T = mat_T.T.flatten().unsqueeze(dim=1)
    mask_T = mask_T.T.flatten().unsqueeze(dim=1).bool()

    return edge_index_T, edge_weight_T, mat_T, mask_T

T = 5
edge_index_T, edge_weight_T, mat_T, mask_T = get_product_graph(mat, mask, T)

In [3]:
X_T = torch.zeros_like(mat[:, :T])
X_T[mask[:, :T]] = mat[:, :T][mask[:, :T]]
X_reconstructed = feature_propagation(edge_index, edge_weight, X_T, mask[:, :T], 40)
norm_mse = torch.norm(X_reconstructed - mat[:, :T]) / torch.norm(mat[:, :T])
print(norm_mse)


tensor(0.1781)


In [4]:
from imputers.feature_propgate import feature_propagation

X_T = torch.zeros_like(mat_T)
X_T[mask_T] = mat_T[mask_T]
X_reconstructed = feature_propagation(edge_index_T, edge_weight_T, X_T, mask_T, 40)
norm_mse = torch.norm(X_reconstructed - mat_T) / torch.norm(mat_T)
print(norm_mse)

tensor(0.1388)


In [14]:
from compare_FP import SpectralLoss
from compare_FP import train_model, test_model
from imputers.my_imputer2 import FPGCN
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)
model = FPGCN(1, 1, 1, 3)
# loss_fn = SpectralLoss(1e-3, laplacian)
loss_fn = nn.MSELoss()

lr = 1e-2

train_model(model, X_T, edge_index_T, edge_weight_T, mask_T, epochs=1000, lr=lr, loss_fn=loss_fn)
X_pred = test_model(model, X_T, edge_index_T, edge_weight_T, mask_T)

norm_mse = torch.norm(X_pred - mat_T) / torch.norm(mat_T)
print(norm_mse)


tensor(0.1366)


In [11]:
mask

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [ ]:
def product_fp(X, adj, mask, K):
    N = X.shape[0]
    edge_index = []
    for i in range(K):
        cur_edge_index = adj + i * N
        edge_index.append(cur_edge_index)

    for i in range(K-1):
        nodes = torch.arange(N) + i * N
        # include nodes if mask
        cur_edge_index = torch.stack([nodes, nodes + N], dim=0)

        edge_index.append(cur_edge_index)

    edge_index = torch.cat(edge_index, dim=1)
    edge_weight = torch.ones(edge_index.shape[1])

    X_hat = torch.zeros_like(X)
    X_hat[mask] = X[mask]

    X_reconstructed = feature_propagation(edge_index, edge_weight, X_hat, mask, 40)
    return X_reconstructed



In [23]:
import pandas as pd
import numpy as np
import torch
from torch import Tensor
from torch_geometric.typing import Adj, OptTensor
from torch_scatter import scatter_add
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import matplotlib.pyplot as plt



from utils.metrics import accuracy

import pandas as pd
import numpy as np
import torch
from torch import Tensor
from torch_geometric.typing import Adj, OptTensor
from torch_scatter import scatter_add
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import matplotlib.pyplot as plt


class FPLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, mask_reverse=False):
        super().__init__()  
        self.lin = Linear(in_channels, out_channels)
        self.bias = Parameter(torch.Tensor(out_channels))
        self.mask_reverse = mask_reverse
        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index, M):
        print("user", self.__user_args__)
        return 
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        x = self.lin(x)
        out = self.propagate(edge_index, x=x, norm=norm) * M + x * (~M)
        if self.mask_reverse:
            out = self.propagate(edge_index, x=out, norm=norm) * (~M) + x * M
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j
    

class FPGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, reverse=True):
        super(FPGCN, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv1 = FPLayer(in_channels, hidden_channels, reverse)
        self.conv2 = FPLayer(hidden_channels, hidden_channels, reverse)

    def forward(self, x, edge_index, M):
        x = self.conv1(x, edge_index, M)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, M)
        return x




def get_symmetrically_normalized_adjacency(edge_index, n_nodes):
    """
    Given an edge_index, return the same edge_index and edge weights computed as
    \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}} \mathbf{\hat{D}}^{-1/2}.
    """
    edge_weight = torch.ones((edge_index.size(1),), device=edge_index.device)
    row, col = edge_index[0], edge_index[1]
    deg = scatter_add(edge_weight, col, dim=0, dim_size=n_nodes)
    deg_inv_sqrt = deg.pow_(-0.5)
    deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float("inf"), 0)
    DAD = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    return edge_index, DAD

class FeaturePropagation(torch.nn.Module):
    def __init__(self, num_iterations: int):
        super(FeaturePropagation, self).__init__()
        self.num_iterations = num_iterations

    def propagate(self, x: Tensor, edge_index: Adj, mask: Tensor) -> Tensor:
        # out is inizialized to 0 for missing values. However, its initialization does not matter for the final
        # value at convergence
        out = x
        if mask is not None:
            out = torch.zeros_like(x)
            out[mask] = x[mask]

        n_nodes = x.shape[0]
        adj = self.get_propagation_matrix(out, edge_index, n_nodes)
        for _ in range(self.num_iterations):
            # Diffuse current features
            out = torch.sparse.mm(adj, out)
            # Reset original known features
            out[mask] = x[mask]

        return out

    def get_propagation_matrix(self, x, edge_index, n_nodes):
        # Initialize all edge weights to ones if the graph is unweighted)
        edge_index, edge_weight = get_symmetrically_normalized_adjacency(edge_index, n_nodes=n_nodes)
        adj = torch.sparse.FloatTensor(edge_index, values=edge_weight, size=(n_nodes, n_nodes)).to(edge_index.device)

        return adj


def feature_propagation(edge_index, X, feature_mask, num_iterations):
    propagation_model = FeaturePropagation(num_iterations=num_iterations)

    return propagation_model.propagate(x=X, edge_index=edge_index, mask=feature_mask)


def run_model(model, X, edge_index, mask, epochs=200, lr=1):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(X, edge_index, mask)
        loss = criterion(out[mask], X[mask])
        loss.backward()
        optimizer.step()
    model.eval()
    out = model(X, edge_index, mask)
    out[mask] = X[mask]
    return out




In [29]:
edge_index = edge_index.to("cuda")
edge_index._indices()

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [26]:
layer = FPLayer(12, 12, True)
layer.flow
layer.__user_args__

{'norm', 'x_j'}

In [85]:
torch.manual_seed(0)
model = FPGCN(1, 1)
out = run_model(model, x, edge_index, mask, epochs=200, lr=0.9)
out[mask] = x[mask]
acc3 = accuracy(out,  mat[:, 0].unsqueeze(1)).item()
print("FPGCN", acc3)

FPGCN 0.9651561975479126


Least squares 

In [220]:
mask_ = mask[:, 0]
def least_square(x, mask_, adj):
    # eigendecomposition
    L = torch.diag(torch.sum(adj, dim=1)) - adj
    # symmetric normalization
    D = torch.diag(torch.pow(torch.sum(adj, dim=1), -0.5))
    L = D @ L @ D
    eigval, eigvec = torch.linalg.eigh(L)

    # B
    num_nodes = x.shape[0]
    k = 2
    B = eigvec[mask_, :k].float()

    # reconstruction
    alpha = torch.linalg.lstsq(B, x[mask_])[0].double()
    x_reconstructed = eigvec[:, :k] @ alpha
    return x_reconstructed
    # accuracy
    
x_reconstructed = least_square(x, mask_, adj)
print("eigendecomp", accuracy(x_reconstructed, mat[:, 0].unsqueeze(1)))


eigendecomp tensor(0.7978, dtype=torch.float64)


Least squares 

In [ ]:

# eigendecomposition
L = torch.diag(torch.sum(adj, dim=1)) - adj
eigval, eigvec = torch.linalg.eigh(L)

# B
num_nodes = x.shape[0]
k = num_nodes 
B = eigvec[~mask, :k].float()

# reconstruction
alpha = torch.linalg.lstsq(B, x[~mask])[0].double()
x_reconstructed = eigvec[:, :k] @ alpha

# accuracy
print("eigendecomp", accuracy(x_reconstructed, mat[:, 0].unsqueeze(1)))

eigendecomp tensor(0.6953, dtype=torch.float64)


POCS

In [57]:
total = []

for i in range(mat.shape[1]):
    model = FPGCN(1, 1)
    x = mat[:, i].clone().unsqueeze(1)
    mask_i = mask[:, i].unsqueeze(1)
    out = run_model(model, x, edge_index,mask_i, epochs=200, lr=0.9)
    out[mask_i] = x[mask_i]
    total.append(out)

total = torch.cat(total, dim=1)
print("FPGCN per node", accuracy(total, mat))

FPGCN per node tensor(0.8326, grad_fn=<RsubBackward1>)


Non bandlimited

In [32]:
S 
gamma = 0.5

FPGCN 0.9045080721378327 0.012402761317306495


In [218]:
accs = []
for _ in range(10):
    model = FPGCN(1, 1, 1, False)

    out, losses = run_model(model, x, edge_index, mask, epochs=200, lr=1)
    acc = accuracy(out,  mat[:, 0].unsqueeze(1)).item()
    accs.append(acc)
print("FPGCN", np.mean(accs), np.std(accs))

FPGCN 0.9021665394306183 0.0013546581267895339


In [219]:
accs = []
for _ in range(10):
    model = FPGCN(1, 1, 1, True)

    out, losses = run_model(model, x, edge_index, mask, epochs=200, lr=1)
    acc = accuracy(out,  mat[:, 0].unsqueeze(1)).item()
    accs.append(acc)
print("FPGCN", np.mean(accs), np.std(accs))

FPGCN 0.9196623623371124 0.017789230962225674


In [201]:
mask.sum()

tensor(23)

10 rounds

In [120]:
def create_table(mat, rate):
    # iterate through each row 
    acc = []
    n_nodes = mat.shape[0]
    # sample 10% of matrix
    for i in range(0, 100, 10):
        x = mat[:, i].clone().unsqueeze(1)
        mask = torch.bernoulli(torch.Tensor([1 - rate]).repeat(n_nodes, 1)).bool()
        x = torch.where(torch.isnan(x), 0, x).float()
        X_reconstructed = feature_propagation(edge_index, x, mask, num_iterations=40)
        acc1 = accuracy(X_reconstructed, mat[:, i].unsqueeze(1)).item()

        x_reconstructed = least_square(x, mask[:, 0], adj)
        acc2 = accuracy(x_reconstructed, mat[:, i].unsqueeze(1)).item()
        
        # acc2 = run_model(model, x, edge_index, mat[0], mask, epochs=200, lr=1).item()
        model = FPGCN(1, 1)
        out = run_model(model, x, edge_index, mask, epochs=200, lr=0.9)
        out[mask] = x[mask]
        acc3 = accuracy(out,  mat[:, i].unsqueeze(1)).item()

        # acc4 = accuracy(x_reconstructed, mat[:, i].unsqueeze(1)).item()
        acc += [[acc1, acc2, acc3]]

    return acc


In [121]:
acc = create_table(mat, 0.1)
df = pd.DataFrame(acc, columns = ['FP', 'LS', 'FPGCN'])
df

,FP,LS,FPGCN
0,0.948195,0.797770,0.962479
1,0.964092,0.792761,0.974164
2,0.923277,0.785871,0.964625
3,0.916618,0.779121,0.957560
4,0.961058,0.788087,0.961842
5,0.937543,0.793000,0.967421
6,0.940785,0.796125,0.958619
7,0.963757,0.789225,0.967597
8,0.904950,0.675237,0.912041
9,0.850842,0.580546,0.857726


In [101]:
acc = create_table(mat, 0.5)
df = pd.DataFrame(acc, columns = ['FP', 'LS', 'FPGCN'])
df

,FP,LS,FPGCN
0,0.887777,0.797800,0.940827
1,0.861391,0.781511,0.921497
2,0.853740,0.785576,0.922116
3,0.859012,0.778907,0.914380
4,0.861217,0.787231,0.904881
5,0.858380,0.781582,0.901208
6,0.884360,0.796130,0.899455
7,0.869189,0.777709,0.940221
8,0.800578,0.675324,0.828502
9,0.739614,0.579251,0.742680


In [103]:
acc = create_table(mat, 0.9)
df = pd.DataFrame(acc, columns = ['FP', 'LS', 'FPGCN'])
df

,FP,LS,FPGCN
0,0.754014,0.786573,0.914383
1,0.760862,0.781521,0.875429
2,0.817785,0.795783,0.896913
3,0.782505,0.772361,0.885320
4,0.768814,0.776174,0.868299
5,0.802326,0.781920,0.879931
6,0.704400,0.776584,0.869892
7,0.765826,0.777228,0.872962
8,0.676791,0.662217,0.750001
9,0.616756,0.572055,0.674827


acc

In [3]:
Inputs = []


In [26]:
import argparse
import multiprocessing

import pandas as pd
import numpy as np
import torch
from torch import Tensor
from torch_geometric.typing import Adj, OptTensor
from torch_scatter import scatter_add
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import torch.nn as nn


from imputers.feature_propgate import feature_propagation
from imputers.my_imputer2 import FPGCN 
from compare_FP import train_model, test_model

DATA_PATHS = {
    "shenzhen": {"feat": "data/sz_speed.csv", "adj": "data/sz_adj.csv"},
    "losloop": {"feat": "data/los_speed.csv", "adj": "data/los_adj.csv"},
}


def create_mask(shape, k):
    np.random.seed(0)

    total = shape[0] * shape[1]
    num_zeros = int(total * k / 100)
    indices = np.random.choice(total, num_zeros, replace=False)
    mask = np.ones((shape[0], shape[1]), dtype=bool)
    mask.ravel()[indices] = 0
    return torch.from_numpy(mask)

def get_product_graph(mat, T):
    N = mat.shape[0]
    mat_T = mat[:, :T].clone()
    nodes = torch.arange(N) 

    edge_index_T = []
    for i in range(T):
        cur_edge_index = edge_index + i * N
        edge_index_T.append(cur_edge_index)

    for i in range(T-1):
        cur_edge_index = torch.stack([nodes + i * N, nodes + (i + 1) * N], dim=0)
        edge_index_T.append(cur_edge_index)
        cur_edge_index = torch.stack([nodes + (i + 1) * N, nodes + i * N], dim=0)
        edge_index_T.append(cur_edge_index)
    
    edge_index_T = torch.cat(edge_index_T, dim=1)
    edge_weight_T = torch.ones(edge_index_T.shape[1])
    mat_T = mat_T.T.flatten().unsqueeze(dim=1)

    return edge_index_T, edge_weight_T, mat_T


class Args:
    data = "shenzhen"
    T = 2
if __name__ == "__main__":
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--data", type=str, default="shenzhen")
    # parser.add_argument("--T", type=int, default=0)
    # args = parser.parse_args()

    args = Args()
    torch.manual_seed(0)
    np.random.seed(0)

    data = args.data
    mat = pd.read_csv(DATA_PATHS[data]["feat"]).to_numpy() 
    adj = pd.read_csv(DATA_PATHS[data]["adj"], header=None).to_numpy()
    mat = torch.tensor(mat).float()
    adj = torch.tensor(adj)
    edge_index = torch.nonzero(adj, as_tuple=False).t().contiguous()
    # edge_weight = torch.ones(edge_index.shape[1])
    edge_weight = adj[edge_index[0], edge_index[1]].float()
    if args.T > 0:
        mat = mat.T[:, :args.T]
    else: 
        args.T = mat.shape[1]
    results = {
        "fp": [],
        # "product fp_gcn": [],
        # "product fp": [],
    }

    loss_fn = nn.MSELoss()
    lr = 1e-1

    rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, .7, .8, .9]
    rates = [0.3, 0.1]
    rates = [0.1]
    mat_og = mat.clone()

    edge_index_T, edge_weight_T, mat_T = get_product_graph(mat, args.T)

    for missing_rate in  rates:
        mask = create_mask(mat.shape, missing_rate * 100).bool()
        X = torch.zeros_like(mat)
        X[mask] = mat[mask].clone()

        X_reconstructed = feature_propagation(edge_index, edge_weight, X, mask, 40) + 1e-2
        print(X_reconstructed[~mask])
        # save inputs in file
     
        rmse = compute_rmse(mat[~mask].numpy(), X_reconstructed[~mask].numpy())
        results["fp"].append(rmse)


        # mask_T = mask.T.flatten().unsqueeze(dim=1)
        # mask_T_og = mask_T.clone()
        # X = torch.zeros_like(mat_T)
        # X[mask_T] = mat_T[mask_T].clone()
        # X_reconstructed = feature_propagation(edge_index_T, edge_weight_T, X, mask_T, 40)
        # norm_mse = torch.norm(X_reconstructed - mat_T) / torch.norm(mat_T)
        # results["product fp"].append(norm_mse)

        # model = FPGCN(1, 1, 1)
        # mask_T = mask.T.flatten().unsqueeze(dim=1)
        # train_model(model, X, edge_index_T, edge_weight_T, mask_T, epochs=1000, lr=lr, loss_fn=loss_fn)
        # X_pred = test_model(model, X, edge_index_T, edge_weight_T, mask_T)

        # norm_mse = torch.norm(X_pred - mat_T) / torch.norm(mat_T)   
        # results["product fp_gcn"].append(norm_mse)

        # assert torch.allclose(mat, mat_og)
        # assert torch.allclose(mat.T.flatten().unsqueeze(dim=1), mat_T)
        # assert torch.allclose(mask_T, mask_T_og)

    for key in results:
        print(key, results[key])
    # plt.title(args.data)
    # plt.plot(np.arange(0.1, 1, 0.1), overall_results["gcn1"], label="GCN1")
    # for key in results:
    #     plt.plot(rates, results[key], label=key)
    #     plt.xlabel("missing rate")
    #     plt.ylabel("Normalized RMSE")

    # plt.legend()
    # plt.savefig(args.data + ".png")
    # save stats


tensor([1.4382e+01, 8.7881e+00, 2.2199e+01, 1.6183e+01, 2.8469e+01, 8.9806e+00,
        2.0555e+01, 5.0092e+01, 1.8176e+01, 4.7541e+01, 1.1540e+01, 8.4829e+00,
        2.7252e+01, 3.1506e+01, 1.5267e+01, 1.1474e+01, 1.6660e+01, 9.7757e+00,
        2.3871e+00, 1.2194e+00, 1.0000e-02, 1.0000e-02, 1.1453e+01, 1.3319e+01,
        1.1816e+01, 1.7837e+01, 2.1117e+01, 2.3383e+01, 3.1340e+01, 4.5562e+00,
        2.5768e+01])
fp [[11.385915231117858, inf]]


/tmp/ipykernel_679264/2473974349.py:7: RuntimeWarning: divide by zero encountered in divide
  return np.sum(np.abs(var - var_hat) / var) / var.shape[0]


In [18]:
dense_mat = mat
mask = create_mask(mat.shape, missing_rate * 100).bool()
sparse_mat = torch.zeros_like(mat)
sparse_mat[mask] = mat[mask].clone()
missing_rate


0.1

In [20]:
import numpy as np
np.random.seed(1000)

import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
        
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - N * T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR_2d(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x


# dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
# for t in range(2, 5):
#     dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                        #   axis = 1)
dim1, dim2 = dense_mat.shape

# missing_rate = 0.5
# sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
dense_mat = dense_mat.numpy()
sparse_mat = sparse_mat.numpy()

x = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))


20
1.0
20.786764323222386

40
1.0
20.786764323222386

60
1.0
20.786764323222386

80
1.0
20.786764323222386

100
1.0
20.786764323222386

Running time: 0 seconds.


In [26]:
mask_1 == mask_2

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True